In [1]:
import pandas as pd 
import numpy as np
import re
import nltk
stemmer = nltk.stem.porter.PorterStemmer()
df = pd.read_csv('tweets.csv')
df[0:10]
print(len(df['text']))

203451


In [2]:
df.dropna(subset=["text"], inplace=True)

retweetedRussianTweets = df.retweeted_status_id.isin(df.tweet_id) & df.retweeted_status_id.notnull()
df2 = df.drop(df[retweetedRussianTweets].index)

df_unique_stemmed = df2.drop_duplicates('text')

r = re.compile(r'RT @\w*:')
retweet_series = df_unique_stemmed.text.str.contains(r)
rt_field_series = df_unique_stemmed.retweeted_status_id.notnull()

df_unique_stemmed[0:10]
# df['text'] = df["text"].apply(lambda x: [stemmer.stem(y) for y in x])
url_regex = re.compile(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)')
# username_regex = re.compile(r'RT @\w*')
username_regex = re.compile(r'@(?=\w)')
hashtag_regex = re.compile(r'#(?=\w)')

df_unique_stemmed.text = (df_unique_stemmed.text.str.replace(r,repl='')
    .str.replace(url_regex,repl='')
    .str.replace(username_regex, repl='')
    .str.replace(hashtag_regex, repl='')
    .str.replace('\n', repl='')
    .str.replace('\r', repl='')       
#    .apply(lambda x: ' '.join([stemmer.stem(y) for y in x.split(' ')]))
                         )

df_unique_stemmed.dropna(subset=["text"], inplace=True)
df_unique_stemmed[~(~retweet_series & ~rt_field_series)]['text'].to_csv('real_stemmed.txt',index=False)
df_unique_stemmed[~retweet_series & ~rt_field_series]['text'].to_csv('russian_stemmed.txt',index=False)


print(len(df_unique_stemmed['text']))
print(len(df_unique_stemmed[~(~retweet_series & ~rt_field_series)]['text']))
print(len(df_unique_stemmed[~retweet_series & ~rt_field_series]['text']))

KeyboardInterrupt: 

In [ ]:
df = pd.read_csv('./tweets.csv')

df.dropna(subset=["text"], inplace=True)

retweetedRussianTweets = df.retweeted_status_id.isin(df.tweet_id) & df.retweeted_status_id.notnull()
df2 = df.drop(df[retweetedRussianTweets].index)

df_unique = df2.drop_duplicates('text')

r = re.compile(r'RT @\w*:')
retweet_series = df_unique.text.str.contains(r)
rt_field_series = df_unique.retweeted_status_id.notnull()

df_unique[0:10]
# df['text'] = df["text"].apply(lambda x: [stemmer.stem(y) for y in x])
url_regex = re.compile(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)')
# username_regex = re.compile(r'RT @\w*')
username_regex = re.compile(r'@(?=\w)')
hashtag_regex = re.compile(r'#(?=\w)')

df_unique.text = (df_unique.text.str.replace(r,repl='')
    .str.replace(url_regex,repl='')
    .str.replace(username_regex, repl='')
    .str.replace(hashtag_regex, repl='')
    .str.replace('\n', repl='')
    .str.replace('\r', repl=''))

df_unique.dropna(subset=["text"], inplace=True)

df_unique[~(~retweet_series & ~rt_field_series)]['text'].to_csv('real.txt',index=False)
df_unique[~retweet_series & ~rt_field_series]['text'].to_csv('russian.txt',index=False)
# df_original["russian"] = 0

print(len(df_unique['text']))
print(len(df_unique[~(~retweet_series & ~rt_field_series)]['text']))
print(len(df_unique[~retweet_series & ~rt_field_series]['text']))

In [ ]:
## Add a column bool_russian which is our target column for the classifier
df_unique['bool_russian'] = (~retweet_series & ~rt_field_series).apply(lambda x: 1 if x else 0)
df_unique[:10][['text','bool_russian']]

In [ ]:
#dfLength = len(df_unique['text'])

## Here we will create out list of stopwords

from nltk.corpus import stopwords

stoplist = stopwords.words('english')
PS3StopWords = ["ever", "one", "do","does","make", "go", "us", "to", "get", "about", "may", "s", ".", ",", "!", "i", "I", '\"', "?", ";", "--", "--", "would", "could", "”", "Mr.", "Miss", "Mrs.", "don’t", "said", "can't", "didn't", "aren't", "I'm", "you're", "they're", "'s"]
stoplist.extend(PS3StopWords)
personalAdditions = ["it", "i'm", "|", "–", "-", "~", "you're", "thing", '"', "…", '…"', ""]
stoplist.extend(personalAdditions)

from gensim import corpora
tweets = [[word for word in tweet.lower().split() if word not in stoplist] for tweet in df_unique['text']]
dictionary = corpora.Dictionary(tweets)

## Here we will remove stop words and words that have a frequency less than 5

from six import iteritems
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist
           if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq < 5]
dictionary.filter_tokens(stop_ids + once_ids)  # remove stop words and words that appear only once
dictionary.compactify()  # remove gaps in id sequence after words that were removed
print(dictionary)
print(tweets[0:10])

In [ ]:
## This following nested for loop creates the raw counts for the feature vectors

print(len(df_unique['text'])) # Number of tweets
print(len(dictionary)) # Length of the feature vectors

featureVectors = np.zeros((len(df_unique['text']),len(dictionary)),int)

for i in range(len(df_unique['text'])) :
    for word in tweets[i]:
        if word in dictionary.token2id :
            word_id = dictionary.token2id[word]
            featureVectors[i][word_id] += 1
            
print(featureVectors[0])

In [ ]:
#import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn import metrics

In [ ]:
npdata = featureVectors
nptarget = np.array(df_unique['bool_russian'])

print("You have ", npdata.shape[0], "training instances")
print("You have ", npdata.shape[1], "features")

In [ ]:
# randomly select a subset of your data (size = 20)
testid = [1, 1]
while len(testid) != len(set(testid)):
    testid = np.random.randint(0, npdata.shape[0], 2000)

# Get your testing data
print(testid)
testset = npdata[testid, :]
testtarget = nptarget[testid]
print(testset.shape)

trainset = npdata
traintarget = nptarget

# Get your training data
#trainset = np.delete(npdata, testid, 0)
#traintarget = np.delete(nptarget, testid, 0)
print(trainset.shape)



In [ ]:
from nltk import FreqDist
import glob
from nltk.corpus import stopwords
import math
import re
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn import metrics


In [ ]:
from sklearn.model_selection import train_test_split

X = featureVectors
y = df_unique['bool_russian']
X_train, X_test, y_train, y_test = train_test_split(X, y)


In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(13,13,13),max_iter=500)
mlp.fit(X_train,y_train)

In [ ]:
### NEURAL NET CLASSIFIER                                                                                       
#clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
#clf.fit(training, traininglabel)
#predicted = clf.predict(testing)
#print("Accruracy of Multi-Layer Perceptron")
#print(metrics.classification_report(testinglabel, predicted))